# 面向对象的高级编程

## 1 使用\__slots\__

1. 给实例绑定任何属性和方法体现了动态语言的灵活性
2. python中所有的函数、变量都可以视为对象，所以类的属性也可以当做函数来使用

In [1]:
# 1. 为实例绑定新的属性及方法
class Student(object):
    pass
s = Student()
s.name = 'Michael'         # 给实例绑定属性
def myprint():
    print('hello world')
s.my_print = myprint       # 这里是把实例的属性赋值为函数对象了，它仍属于实例的属性，而不是方法
print('1.实例绑定属性的结果:', s.__dict__)

def set_age(self, age):
    self.age = age
from types import MethodType
s.set_age = MethodType(set_age, s)   # 给实例绑定一个方法，其实还是属性，用__dict__仍能看到
s.set_age(25)
print('2.绑定方法后的结果:')
print('  新增了属性age:', s.age)
print('  当前所有属性:', s.__dict__)  # __dict__只输出属性，不包括类中定义的方法

1.实例绑定属性的结果: {'name': 'Michael', 'my_print': <function myprint at 0x000001D6436E2D90>}
2.绑定方法后的结果:
  新增了属性age: 25
  当前所有属性: {'name': 'Michael', 'my_print': <function myprint at 0x000001D6436E2D90>, 'set_age': <bound method set_age of <__main__.Student object at 0x000001D6437031D0>>, 'age': 25}


给特定实例绑定的属性及方法，对该类创建的其他实例是无效的
- 解决方法：给类绑定属性及方法

In [2]:
# 2. 给类绑定属性及方法
class Student(object):       # 新建的类
    pass
def set_score(self, score):  # 需动态增加的方法
    self.score = score
def myprint(self):           # 注意与实例绑定属性中的函数对比，需要多加一个self
    print('hello world!')
Student.set_score = set_score
Student.myprint = myprint
s1 = Student()
s2 = Student()
s1.set_score(97)       # 类绑定方法后，所以实例都可以调用
s2.set_score(99)
print('1.类绑定方法的结果:', s1.score, s2.score)
s1.myprint()

1.类绑定方法的结果: 97 99
hello world!


小结:
- 在实例和类绑定方法时，都可以使用`XXX.attribute_name = func_name`，但在类中添加的方法函数第一个参数必须为self(其实换个名称也行，但为了与类定义对应，使用self)

使用\__slots\__限制实例的属性

In [3]:
# 3. 使用__slots__限制属性名称
class Student(object):
    __slots__ = ('name', 'age')    # 用tuple定义允许绑定的属性名称
s = Student()
s.name = 'Michael'      # 为实例绑定新属性
s.age = 25
# s.score = 100         # score没有再__slots__中，所以会报错

def set_score(self, score):  # 需动态增加的方法
    self.score = score
Student.set_score = set_score       # 给类绑定方法及属性，不受__slots__限制，但实例不行
# Student.set_score(Student, 99)    # 给自己绑定属性
Student.score = 99                  # 只读，实例中不可以修改
# Student.set_score(3)
s = Student()
print('1.类绑定属性及方法不受__slots__限制:', s.score)
# s = Student()
# s.score

1.类绑定属性及方法不受__slots__限制: 99


In [4]:
# __slots__定义的属性对当前实例起作用，对继承的子类不起作用，除非子类中也加入__slots__
class GraduateStudent(Student):
    __slots__ = ('score')        # 加入该句后，才能限制类属性名称，同时可使用父类的__slots__
    pass
g = GraduateStudent()
g.score = 9999
g.name = 'd'
# g.ss = 'f'         # 该句报错，原因父类及子类的__slots__中都没有ss属性

小结:
- 类中绑定属性及方法不受\__slots\__的限制，只有实例才受限制
- 子类中同样使用\__slots\__，才能起到限制属性作用，限制范围:父类+子类的范围

## 2 使用@proerty

### 2.1回顾装饰器的用法

In [5]:
# 1. 使用装饰器为函数动态的添加功能，例如函数执行前，打印日志
import functools   
import datetime

def log(text):             # text参数表示准备输出的信息
    def decorator(func):   # func需要修饰的函数，这里调用hello即可
        @functools.wraps(func)       # 保证修饰后的函数名称保持不变
        def wrapper(*args, **kw):   # 可变参数保证待修饰函数正常运行，该部分是添加额外功能的地方
            now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # 获取当前日期
            print('{0} Current date: {1}'.format(text, now))             # 添加的功能部分
            return func(*args, **kw)      # 执行func函数并返回
        return wrapper                    # 返回装饰后的函数
    return decorator                      # 返回装饰器

@log('1.这是修饰器添加的部分:')             # 在函数定义的前面加
def hello():     # 为该函数添加装饰器，增加输出当前日期的功能
    print('2.这是函数本身的输出: Hello world! Hello python!')
print('---------------------------------------------------')
hello()          # 调用装饰后的函数

---------------------------------------------------
1.这是修饰器添加的部分: Current date: 2018-08-12 13:47:00
2.这是函数本身的输出: Hello world! Hello python!


### 2.2 @property装饰器将方法变成属性调用
即使用属性调用的方式，同时可检查参数的合理性

In [6]:
# 2. 属性修饰器方便又好用
class Student(object):
    @property
    def score(self):         # 1
        return self._score   # _score名称可以改变(属性名之外名称)，但约定俗成，属性名前加单个下划线
    @score.setter            # 2
    def score(self, value):  # 3 ，其中1,2,3处的score名称一定要统一
        if not isinstance(value, int):
            raise ValueError('Score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('Score must between 0 ~ 100!')
        self._score = value
s = Student()
s.score = 99
print('1.使用属性修饰器的结果:', s.score)
# s.score = 1000     # 检查属性的合理性，不合理会报错

1.使用属性修饰器的结果: 99


小结:
- @property 属性装饰器把getter方法变为属性
- @attribute_name.setter 把setter方法变为属性赋值，如果没有该步骤，则属性变为只读

属性装饰器练习

In [7]:
# @property给一个Screen对象加上width和height属性，以及一个只读属性resolution
class Screen(object):
#     def __init__(self):      # 初始化省略也可以
#         self._width = 0
#         self._height = 0
#         self._resolution = 0
    @property                # 添加width属性
    def width(self):
        return self._width
    @width.setter            # 使width属性可修改
    def width(self, value):
        if not isinstance(value, int):
            raise ValueError('Value must be integer!')
        else:
            self._width = value
        
    @property                # 添加height属性
    def height(self):
        return self._height
    @height.setter
    def height(self, value):
        if not isinstance(value, int):
            raise ValueError('Value must be integer!')
        else:
            self._height = value
    
    @property
    def resolution(self):    # 该属性只读
        return self._width * self._height

s = Screen()
s.width = 1024
s.height = 768
print('1.可读写属性:', s.width, s.height)
print('2.只读属性resolution:', s.resolution)   

1.可读写属性: 1024 768
2.只读属性resolution: 786432


## 3 多重继承

一个子类可以继承多个父类，举例Dog、Bat、Parrot和Ostrich

In [8]:
class_str="""
                Animal
        __________|__________
       |                     |
    Runnable              Flyable
  _____|_____           _____|_____ 
 |           |         |           |
Dog       Ostrich   Parrot        Bat
"""
print('分类结果:', class_str)

分类结果: 
                Animal
        __________|__________
       |                     |
    Runnable              Flyable
  _____|_____           _____|_____ 
 |           |         |           |
Dog       Ostrich   Parrot        Bat



在多重继承中，每种子类都有一个主线继承的父类，其他继承的父类是额外"混入"的，这种方式成为MixIn模式

In [9]:
# 动物分类
class Animal(object):   # 最基本的动物类
    pass

# 子类要继承的主线类
class Mammal(Animal):   # 从Animal类继承来的主要类
    pass
class Bird(Animal):
    pass

# 各种动物
class Dog(Mammal):     # 基本的继承关系
    pass
class Bat(Mammal):
    pass
class Parrot(Bird):
    pass
class Ostrich(Bird):
    pass

In [10]:
# 需要添加额外的父类，进行多重继承
class RunnableMixIn(object):    # 定义额外的父类
    def run(self):
        print('Running...')
class FlyableMixIn(object):
    def fly(self):
        print('flying...')
class Dog(Mammal, RunnableMixIn):  # 即继承了Mammal，同时继承了RunnableMixIn
    pass

多重继承练习

In [11]:
# 使用DAG，有向无环图，c3算法来实现多重继承
class A(object):          #定义两个基类
    def foo(self):
        print('A foo')
    def bar(self):
        print('A bar')
class B(object):
    def foo(self):
        print('B foo')
    def bar(self):
        print('B bar')
class C1(A, B):          # 对A和B进行了多重继承
    def foo(self):
        print('C1-foo')
class C2(A, B):
    def foo(self):
        print('C2-foo')

class D(C1, C2):         # D对C1和C2进行了多重继承
    pass
if __name__ == '__main__':
    print('1.显示继承的顺序:', D.__mro__)     # 显示继承的顺序
    d = D()
    print('2.调用父类方法:沿着继承的顺序，从左边起先出现具有该方法的父类，然后调用该方法:')
    d.foo()
    d.bar()

1.显示继承的顺序: (<class '__main__.D'>, <class '__main__.C1'>, <class '__main__.C2'>, <class '__main__.A'>, <class '__main__.B'>, <class 'object'>)
2.调用父类方法:沿着继承的顺序，从左边起先出现具有该方法的父类，然后调用该方法:
C1-foo
A bar


小结:
- 多重继承组合多个MixIn的功能，即拥有所有已继承父类的属性及方法

## 4 定制类

\__XXX\__变量相关，该类变量有特殊用途

###  1. \__str\__
输出实例的信息

In [12]:
# 该字符串是记录实例信息的
class Student(object):
    def __init__(self, name):
        self.name = name
s = Student('Mike')
print('1.实例的信息:', s)   # 该信息与__str__字符串有关 
class Student(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'Student object (name: %s)' % self.name
s1 = Student('David')
print('2.重定义__str__()方法后的结果:', s1)
s1   # 但直接输出该实例信息，使用的是改变前的结果，这样调用的是__repr__

1.实例的信息: <__main__.Student object at 0x000001D643747940>
2.重定义__str__()方法后的结果: Student object (name: David)


In [13]:
# 为了使两种输出的结果都改变，可以将__repr__重写，但该方法与__str__一致，所以直接将__str__函数对象
# 指向__repr__即可
class Student(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):    # 重定义而改变实例输出的信息
         return 'Student object (name: %s)' % self.name
    __repr__ =  __str__   # 将__str__函数对象指向__repr__，相当于重定义了相同的该函数
s = Student('David')
print('2.重定义__str__()方法后的结果:', s)
s   # 但直接输出该实例信息，使用的是改变前的结果，这样调用的是__repr__

2.重定义__str__()方法后的结果: Student object (name: David)


Student object (name: David)

### 2. \__iter\__
与\__next\__方法一起实现对象的可迭代，可以使用for循环遍历，如list或tuple

In [14]:
# 实现斐波那契数列
class Fib(object):
    def __init__(self):
        self.a, self.b = 0, 1   # 初始化前两个数
    def __iter__(self):
        return self             # 实例本身是迭代对象，所以返回自己
    def __next__(self):
        self.a, self.b = self.b, self.a + self.b   # 计算下一个值
        if self.a > 1000:
            raise StopIteration('The loop is stopped.')   # 为什么输出不提示超出范围？？？？
        return self.a
fib = Fib()
for i in fib:
    print(i, end=' ')

1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 

### 3. \__getitem\__
获取指定下标的元素

In [15]:
# 1. 使用__getitem__可以访问指定下标元素
class Fib(object):
    def __getitem__(self, n):   # 该方法使实例可以使用f[index]的方式访问，但是只读的
        self.a, self.b = 1, 1
        for x in range(n):
            self.a, self.b = self.b, self.a + self.b
        return self.a
#     def __setitem__(self, key, value):    # f[1] = 2333功能没有实现？？？？？？
# #         self[key] = value
# #         print(self._mylist)
#         self._mylist[key] = value
#         print('new key:', self[key], type(self[key]), key, value, self)

f = Fib()
print('1.获取指定下标的元素:', end=' ')
for i in range(10):
    print(f[i], end=' ')
# print()
# f[1] = 2333
# print(f[2])
# print('***')

1.获取指定下标的元素: 1 1 2 3 5 8 13 21 34 55 

In [16]:
# 2. 添加切片访问的功能
class Fib(object):
    def __getitem__(self, n):
        if isinstance(n, int):       # 下标的访问形式
            self.a, self.b = 1, 1
            for x in range(n):
                self.a, self.b = self.b, self.a + self.b
            return self.a
        if isinstance(n, slice):  # 切片的访问形式
            start = n.start if n.start else 0   # 获取切片的起始和终止位置, f[:7]这种也可以正常访问
            stop  = n.stop
            self.a, self.b = 1, 1
            self.L = []
            for x in range(stop):               # 循环迭代创建序列
                if x >= start:
                    self.L.append(self.a)
                self.a, self.b = self.b, self.a + self.b
            return self.L        
f = Fib()
print('1.获取指定下标的元素:', end=' ')
for i in range(10):
    print(f[i], end=' ')
print('\n2.切片的访问方法:', f[:10])

1.获取指定下标的元素: 1 1 2 3 5 8 13 21 34 55 
2.切片的访问方法: [1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


### 4. \__getattr\__
可以动态返回一个属性，即还没有定义的属性，当调用不存在的属性时调用次方法来获取属性

In [17]:
# 动态返回属性
class Student(object):
    def __init__(self, name):
        self.name = name
    def __getattr__(self, attr):    # 当输出name外的属性时，调用该方法
        if attr == 'score':         # 如果输出的属性时score，则返回100，score，name外属性返回None
            return 100              # 返回值
        if attr == 'age':
            return lambda : 25      # 返回函数
#         raise AttributeError("Student object has no attribute %s" % attr)  # 可以抛出异常
s = Student('Mike')
print('1.实例已有的属性:', s.name)
print('2.实例动态输出没有定义的属性score:', s.score)
print('3.实例动态输出没有定义的属性age:', s.age())        # 调用方式不同，多个括号
print('4.既无定义也不在动态输出范围的属性则返回:', s.abc)

1.实例已有的属性: Mike
2.实例动态输出没有定义的属性score: 100
3.实例动态输出没有定义的属性age: 25
4.既无定义也不在动态输出范围的属性则返回: None


In [18]:
# 链式动态调用url
class Chain(object):
    def __init__(self, path=''):
        self._path = path
    def __getattr__(self, path):   # 动态调用
        return Chain('%s/%s' % (self._path, path))
    def __str__(self):
        return self._path
    __repr__ = __str__
out = Chain().status.user.timeline.list     # 调用__str__方法输出类的信息
print('1.动态链式调用的结果:', out)

1.动态链式调用的结果: /status/user/timeline/list


### 5. \__call\__
含有该方法的类的实例可以直接当做函数调用，达到()调用的结果，相当于重载了括号运算符

In [19]:
# 直接调用实例，一般调用实例方式:instance.method(parameters)
class Student(object):
    def __init__(self, name):
        self.name = name
    def __call__(self, age):    # 可以带参数
        print('My name is %s, my age is %s' % (self.name, age))
s = Student('Mike')
print('1.实例本身作为方面调用:', end=' ')
s(25)
print('2.有__call__方法的类定义的实例可调用:', callable(s))

1.实例本身作为方面调用: My name is Mike, my age is 25
2.有__call__方法的类定义的实例可调用: True


动态调用练习

In [20]:
# 动态调用特性，类的所有属性和方法动态化处理
class Chain(object):
    def __init__(self, path=''):
        self._path = path
    def __getattr__(self, path):   # 动态属性调用
        print('call __getattr__(%s)' % path)
        return Chain('%s/%s' % (self._path, path))
    def __str__(self):             # 输出实例的信息
        return self._path
    def __call__(self, param):
        print('call __call__(%s)' % param)
        return Chain('%s/%s' % (self._path, param))
    __repr__ = __str__
print('1.动态链式调用的结果:')
out = Chain().status.user.timeline.list     # 调用__str__方法输出类的信息
print(out)
print('2.动态链式调用的结果:')
out = Chain().users('Mike').repos           # 调用__str__方法输出类的信息
print(out)

1.动态链式调用的结果:
call __getattr__(status)
call __getattr__(user)
call __getattr__(timeline)
call __getattr__(list)
/status/user/timeline/list
2.动态链式调用的结果:
call __getattr__(users)
call __call__(Mike)
call __getattr__(repos)
/users/Mike/repos


## 5 枚举类
枚举类型属于同一个类，每个常量是类的唯一实例，方便管理相同类型的常量数据

In [21]:
# 保存月份
from enum import Enum
# 使用枚举类定义变量，定义Month类型的枚举类，value属性自动赋值，默认从1开始
Month = Enum('Month', ('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', \
                       'Sep', 'Oct', 'Nov', 'Dec'))
for name, member in Month.__members__.items():
    print(name, '=>', member, '', member.value)

Jan => Month.Jan  1
Feb => Month.Feb  2
Mar => Month.Mar  3
Apr => Month.Apr  4
May => Month.May  5
Jun => Month.Jun  6
Jul => Month.Jul  7
Aug => Month.Aug  8
Sep => Month.Sep  9
Oct => Month.Oct  10
Nov => Month.Nov  11
Dec => Month.Dec  12


In [22]:
# Enum派生类更精确地控制枚举类型
from enum import Enum, unique

@unique              # unique装饰器保证数据无重复
class Weekday(Enum):
    sun = 0
    Mon = 1
    Tue = 2
    Wed = 3
    Thu = 4
    Fri = 5
    Sat = 6
day1 = Weekday.Mon
print(day1, day1.value)
print(Weekday['Tue'])
for name, member in Weekday.__members__.items():
    print(name, '=>', member,'',member.value)

Weekday.Mon 1
Weekday.Tue
sun => Weekday.sun  0
Mon => Weekday.Mon  1
Tue => Weekday.Tue  2
Wed => Weekday.Wed  3
Thu => Weekday.Thu  4
Fri => Weekday.Fri  5
Sat => Weekday.Sat  6


枚举练习

In [23]:
# 把Student的gender属性改造为枚举类型，可以避免使用字符串
from enum import Enum
class Gender(Enum):     # 派生Gender枚举类
    Male = 0
    Female = 1
class Student(object):
    def __init__(self, name, gender):
        self.name = name
        self.gender = gender
    def print_info(self):
        print('My name is {}, my gender is {}'.format(self.name, self.gender))
s = Student('David', Gender.Male)    # 没有使用字符串
s.print_info()

My name is David, my gender is Gender.Male


## 6 元类

type()函数可以创建类
1. class的名称
2. 继承的父类集合，注意Python支持多重继承，如果只有一个父类，别忘了tuple的单元素写法
3. class的方法名称与函数绑定，这里我们把函数fn绑定到方法名hello上

In [24]:
# type定义类
def fn(self, name='world'):
    print('Hello, %s' % name)
Hello = type('Hello', (object,), dict(hello=fn))
h = Hello()   # 实例化
h.hello()     # 调用函数
print(type(Hello))
print(type(h))

Hello, world
<class 'type'>
<class '__main__.Hello'>


**metaclass可以创建类**
该部分不常用，所以直接略过